In [20]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as Transformer
from torch.utils.data import DataLoader
import torchvision
import PIL
import os
import numpy as np

In [21]:
train_data = pd.read_csv("data/trainLabels.csv")
train_data.shape

(6283, 2)

In [22]:
train_data

,ID,Class
0,1,n
1,2,8
2,3,T
3,4,I
4,5,R
...,...,...
6278,6279,N
6279,6280,I
6280,6281,P
6281,6282,N


In [23]:
print(train_data["Class"].unique(),len(train_data["Class"].unique()))

['n' '8' 'T' 'I' 'R' 'W' 'L' 'l' 'K' 'G' 'A' 'C' 'u' 't' 'N' 'O' 'k' 'D'
 'w' 'E' 'H' 'e' 'i' 'r' 'a' '0' '1' '6' 'o' 'c' 'S' '2' 'Y' 'Z' 'f' 'J'
 'M' '5' 'd' '7' 'P' 'F' 'j' 'b' 'q' 'X' '3' '4' 'U' '9' 'x' 'm' 'g' 'z'
 'v' 'p' 'B' 'h' 'y' 's' 'V' 'Q'] 62


In [24]:
directory = r"C:\Users\hany5\OneDrive\Desktop\coding stuff\data\deeplearning\c_vision\google characters\data"

In [25]:
train_dir = os.path.join(directory,"trainResized")
test_dir = os.path.join(directory,"testResized")
train_dir

'C:\\Users\\hany5\\OneDrive\\Desktop\\coding stuff\\data\\deeplearning\\c_vision\\google characters\\data\\trainResized'

In [26]:
train_data.shape

(6283, 2)

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_data['Class'])
train_data['new_class'] = le.transform(train_data['Class'])

In [28]:
train_data

,ID,Class,new_class
0,1,n,49
1,2,8,8
2,3,T,29
3,4,I,18
4,5,R,27
...,...,...,...
6278,6279,N,23
6279,6280,I,18
6280,6281,P,25
6281,6282,N,23


In [48]:
transform = Transformer.Compose([Transformer.ToTensor(),torchvision.transforms.Grayscale()])

In [92]:
test_data = []

for file in sorted(os.listdir(test_dir)):
    img=PIL.Image.open(os.path.join(test_dir,file))
    img = transform(img)
    test_data.append(img)

In [95]:
len(test_data)

6220

In [96]:
stupid_test = ['6000.Bmp','10000.Bmp']
print(sorted(stupid_test))

['10000.Bmp', '6000.Bmp']


In [100]:
result_id = sorted(os.listdir(test_dir))

In [50]:
train_data_list = []
count = 0
for i in range(6283):
    file = f"{i+1}.Bmp"
    img=PIL.Image.open(os.path.join(train_dir,file))
    img = transform(img)
    train_data_list.append((img,train_data["new_class"][count]))
    count+=1

In [51]:
train_data_list[0][0].shape

torch.Size([1, 20, 20])

train_data_list == data as tensors 'RGB' in trainResized with their labels
and 
test_data == data as tensors 'RGB' in testResized without labels(labels to predict)


In [52]:
image_size = 20*20

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.block1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=50,kernel_size=5,padding=2),
        nn.ReLU(),
        nn.BatchNorm2d(50),
        nn.Conv2d(in_channels=50,out_channels=50,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(50),
        nn.MaxPool2d(kernel_size=2,stride=2)
        )
        #output is (10,10,50)
        self.block2 = nn.Sequential(
        nn.Conv2d(in_channels=50,out_channels=100,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(100),
        nn.Conv2d(in_channels=100,out_channels=100,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(100),
        nn.MaxPool2d(2,2)
        )
        #output is (5,5,100)
        self.fc1 = nn.Linear(100*5*5,100)
        self.fc2 = nn.Linear(100,62)
    
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.size(0),-1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [168]:
model = torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained = False)
model.conv1 =nn.Conv2d(1,64,(7,7),(2,2),(3,3),bias=False)
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=1000, bias=True),
    nn.Linear(1000,62)
)
model.eval()
model.to(device)

Using cache found in C:\Users\hany5/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\hany5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hany5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [169]:
optimizer = torch.optim.Adam([parameters for parameters in model.parameters() if parameters.requires_grad],lr=0.001)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [170]:
training_data, validation_data = torch.utils.data.random_split(train_data_list,[5000,1283])

In [171]:
train_loader = DataLoader(dataset=training_data,batch_size=50)
validation_loader = DataLoader(dataset=validation_data,batch_size=50)

In [173]:
n_epochs = 100
N_test = len(validation_data)
train_loss = []

for epoch in range(n_epochs):
    model.train()
    batchloss = []
    for x,y in train_loader:
        x,y  = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        y = y.type(torch.int64)
        loss = criterion(y_pred,y)
        loss.backward()
        optimizer.step()
        batchloss.append(loss.item())
    train_loss.append(np.mean(batchloss))
    correct = 0
    print(f"Trainin loss: {train_loss[epoch]}")
    model.eval()
    for x_test,y_test in validation_loader:
        x_test,y_test = x_test.to(device),y_test.to(device)
        y_pred = model(x_test)
        yhat = y_pred.data.max(1,keepdim=True)[1]
        correct +=yhat.eq(y_test.data.view_as(yhat)).sum().item()
    print(f"Epoch: {epoch}, Accuracy: {correct/N_test}")
        

Trainin loss: 0.17657059986144305
Epoch: 0, Accuracy: 0.6531566640685893
Trainin loss: 0.15282385170459747
Epoch: 1, Accuracy: 0.676539360872954
Trainin loss: 0.11526962701231241
Epoch: 2, Accuracy: 0.6336710833982853
Trainin loss: 0.1386922728549689
Epoch: 3, Accuracy: 0.6391270459859704
Trainin loss: 0.29222704205662015
Epoch: 4, Accuracy: 0.5416991426344505
Trainin loss: 1.178833595365286
Epoch: 5, Accuracy: 0.6048324240062354
Trainin loss: 0.44241070717573167
Epoch: 6, Accuracy: 0.6531566640685893
Trainin loss: 0.1600255802925676
Epoch: 7, Accuracy: 0.6858924395946999
Trainin loss: 0.07812796937301755
Epoch: 8, Accuracy: 0.6742010911925176
Trainin loss: 0.057251560925506054
Epoch: 9, Accuracy: 0.6687451286048324
Trainin loss: 0.04345739500597119
Epoch: 10, Accuracy: 0.6804364770070148
Trainin loss: 0.02866260413546115
Epoch: 11, Accuracy: 0.6788776305533905
Trainin loss: 0.028371454434236512
Epoch: 12, Accuracy: 0.6819953234606392
Trainin loss: 0.02264927116339095
Epoch: 13, Accura

In [174]:
result = pd.DataFrame()

In [179]:
result_list = []
for i in test_data:
    i = np.reshape(i,(1,1,20,20))
    i = i.to(device)
    prediction = model(i)
    label = prediction.data.max(1,keepdim=True)[1]
    label = le.inverse_transform(label.cpu())
    label = label.tolist()[0]
    result_list.append(label)


C:\Users\hany5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hany5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hany5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chang

In [180]:
result['ID'] = result_id
result['Class'] = result_list

In [181]:
result['ID'] = result['ID'].str[:-4]

In [182]:
result.to_csv('submission.csv',index=False)